<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/m%24_office_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 辦公室檔案機器人

In [ ]:
!pip install langchain langchain_community langchain_openai rich --quiet

In [ ]:
# 匯入套件和金鑰
from google.colab import userdata
from rich import print as pprint
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### 使用 RAG 處理資料

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 41.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
chat_model = ChatOpenAI()
str_parser = StrOutputParser()
template = (
    "請根據以下內容加上自身判斷回答問題:\n"
    "{context}\n"
    "問題: {question}"
    )
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
!curl https://flagtech.github.io/F4763/example_excel.xlsx -o example_excel.xlsx
!curl https://flagtech.github.io/F4763/example_ppt.pptx -o example_ppt.pptx
!curl https://flagtech.github.io/F4763/example_word.docx -o example_word.docx
!curl https://flagtech.github.io/F4763/example_csv.csv -o example_csv.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15565  100 15565    0     0  47235      0 --:--:-- --:--:-- --:--:-- 47310
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5760k  100 5760k    0     0   9.8M      0 --:--:-- --:--:-- --:--:--  9.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25352  100 25352    0     0  79678      0 --:--:-- --:--:-- --:--:-- 79723
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1976  100  1976    0     0   9939      0 --:--:-- --:--:-- --:--:--  9979


由於 Colab 預先安裝的 NLTK 套件版本是 3.8.1, 但是 unstructured 套件需要使用新版的 NLTK 套件, 所以要先更新套件版本, 才不會出錯。

In [ ]:
# unstructured 需要新版 3.9.1 的 nltk, Colab 預先安裝的是 3.8.1
!pip install --upgrade nltk
!pip install unstructured python-docx python-pptx -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.2 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
from langchain_community.document_loaders import (
    UnstructuredFileLoader)

In [ ]:
def office_file(file_path):
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load()
    return docs

### PowerPoint

In [ ]:
ppt_docs = office_file('/content/example_ppt.pptx')
pprint(ppt_docs)

<ipython-input-17-58840717bebd>:2: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(file_path)


[
    Document(
        metadata={'source': '/content/example_ppt.pptx'},
        page_content='立法院第10屆第3會期經濟委員會\n\n國家發展委員會\x0b業務報告\n\n國家發展委員會\n\n主任委員 
龔明鑫\n\n\n\n全球經濟恢復成長\n\n國際機構全球經濟成長率預測\n\n單位：%\n\n2021年1月預測\n\n2021年2月預測\n\n2021年
3月預測\n\n經濟風險仍高：疫情尚難緩和、美中衝突長期化、全球債務\x0b攀升金融風險升高、不均衡復甦\n\n\n\n2021年展望\n
\n實質輸出成長+5.05%\n\n全球貿易量回升\n\n數位轉型商機\n\n新興科技應用需求仍殷\n\n2021年(f)  
(+4.64%)\n\n單位：%\n\n近7年新高!!\n\n實質民間消費成長+3.74%\n\n國內就業市場穩定\n\n國內外股市活絡\n\n2020年(p)  
(+3.11%)\n\n實質固定投資成長+3.50%\n\n半導體、5G及綠能投資\n\n臺商回臺三大方案\n\n註： 
(p)表初步統計數，(f)表預測數。\n\n資料來源：行政院主計總處，2021年2月20日。\n\n\n\n施政方向與重點\n\n創新導向，\n\n
成為全球經濟關鍵力量\n\n延攬精英，\n\n成為國際人才匯集中心\n\n精進治理，\n\n落實區域永續平衡發展\n\n數位政府，\n\n
提升全方位施政效能\n\n\n\n創新導向，成為全球經濟關鍵力量\n\n\n\n1\n\n協調推動六大核心戰略產業\n\n本會、經濟部、科技
部等18個單位共同推動，\x0b讓臺灣成為全球經濟的關鍵力量\n\n\n\n世界經濟論壇(WEF)107、108連續兩年評比\n\n亞洲‧矽谷1.0
重要成果\n\n2-1\n\n臺灣全球4大超級創新國之一\n\n積極投資新創\n\n物聯網成為兆元產業\n\n加碼匡列\n\n20億→50億\n\n已通
過131家企業， 
\n\n投資19.28億、\n\n帶動投資50.48億元\n\n形塑國家新創品牌\n\n如亞矽執行中心已協助思科於桃園設立智慧創新應用示範中
心\n\n通過181案、投資29.4億\n\nStartup Island TAIWAN於CES、Meet 
Taipei等大型國際展曝光\n\n引進國際大廠創新研發資源\n\n與經濟部共同推動智慧城鄉有感服務\n\n活絡人才及法規調適\n\nAI
篩檢及早治療\n\n5種眼疾辨識\n\n精準度達85%\n\n路邊停車位\n\n尋位時間 
\n\n減少5-8分\n\nAi客製化\n\n英語課程\n\n提升雙語競爭力\n\n空汙AI監測\n\n環保稽核效能 
\n\n提高6倍\n\n交通\n\n停車\n\n健康\n\n醫療\n\n環境\n\n品質\n\n223項\n\n智慧解決方案\n\n教育\n\n學習\n\n出遊旅運\n\
n國道60%路線\n\n免排隊購票\n\n運用無人機 \n\n農耕效率 
\n\n提升10倍\n\n新創法規調適平臺已釐清34項法規議題，如透過平臺業者 
銷售旅遊及健康保險等\n\n已核發2,243張就業金卡，首張YouTube創辦人陳士駿已回台協助新創發展\n\n農畜\n\n漁牧\n\n觀光\n\
n零售\n\n導入全台22縣市\n\n7\n\n\n\n亞洲‧矽谷2.0規劃方向\n\n2-2\n\n亞洲‧矽谷2.0\n\n(2021-2024年)\n\n1.1  
加速AIoT關鍵技術研發\n\n1.2  推動國產化5G開放網路\n\n1.3  強化5G、AI數位科技應用\n\n1.4  促成智慧城鄉跨域合作\n\n  
擴大AIoT\n\n       科技應用\n\n願景\n\n策略1\n\n打造台灣成為亞洲數位創新的關鍵力量\n\n2.1  
擴大投融資加速新創成長\n\n2.2  形塑國家新創品牌強化商機拓展\n\n2.3  促成多元出場帶動正向循環\n\n       精進新創\n\n
發展環境\n\n策略2\n\n3.1  深化物聯網國際夥伴關係\n\n3.2  打造國際級亞矽創新聚落\n\n3.3  促進AIoT輸出海外市場\n\n   
匯聚系統\n\n     輸出能量\n\n策略3\n\n2025 
IoT產值\n\n占全球比重\n\n5%\n\nAIoT解決\n\n方案海外輸出\n\n20案\n\n促成新創出場\n\n40案\n\n培育國際級\n\n系統整合公
司\n\n3家\n\n促成新創\n\n事業成功\n\n200家\n\n目標\n\n8\n\n\n\n3\n\n強化產業及金融鏈結\n\n國家融資保證機制\n\n保證
對象\n\n行政院109年11月核定「國家融資保證機制推動方案」\n\n由國發基金與銀行共同提供專款，提供融資計畫最高6成保證，
由輸銀擔任執行單位，促進金融機構參與綠能等 
重大經濟建設\n\n110年1月輸銀成立「國家融資保證中心」，將協助業者及銀行申請\n\n綠能建設\n\n綠能設備與服務\n\n重大公
共建設\n\n促進私募股權基金投資產業\n\nPE\n\n規劃以私募股權基金專業知識結合國內民間資金， 
投入六大、5+2、前瞻基礎建設及具升級或轉型需求之產業，以促進重要策略性產業發展\n\n研訂促進私募股權基金投資產業輔導要
點(草案)，   規範PE資格、投資產業面向，公私協力認定機制\n\n民間資金\n\n六大\n\n5+2\n\n 前瞻\n\n 
升級轉型\n\n\n\n國發基金投融資計畫\n\n透過多元投資及融資方式，協助企業取得營運發展所需資金\n\n強化產業數位轉型，要
求投資事業提出數位轉型計畫\n\n投資業務 投資件數 總投資金額(億元) 直接投資 62 619.48 投資創業投資事業 53 113.37 
專案投資 280 78.59 創業天使投資方案 131 19.18 投資新創特別股專案 181 29.41\n\n貸款項目 累計核貸件數 
累計核貸金額(億元) 歡迎台商回台投資專案貸款 159 2,786.14 根留台灣企業加速投資專案貸款 66 613.19 海外投資融資貸款 
102 209.66 其他協助產業政策等貸款 3,961 
817.12\n\n\n\n推展數位經濟國際鏈結\n\n4\n\n高階政策對話平台\n\n臺美數位經濟論壇(DEF)\n\n臺歐盟數位經濟對話(DDE)\n\n
本會與歐盟資通訊網絡暨科技    
總署共同舉辦\n\n第1屆會議(2019年布魯塞爾)\n\n會議主題：總體數位政策、產業數位轉型及人工智慧、數位科技基礎建設、數位
技能與工作、數位治理、GDPR、金融科技\n\n第2屆會議(2020年12月視訊會議)\n\n會議主題：總體數位經濟發展政策、研究與科技
合作、區塊鏈、人工智慧、資安產業認證、資料經濟、歐亞數位連結\n\n未來將持續召開第3屆\n\n本會與美國在台協會(AIT)共同
籌辦，已輪流在台美舉辦3屆會議，於DEF架構下規劃合作項目\n\n五大合作面向:\n\n數位貿易與投資\n\n創新創業\n\n智慧技術(
智慧城市、IoT及FinTech)\n\n網路安全\n\n全球合作\n\n第4屆由美方主辦，規劃於2021年召開\n\n\n\n延攬精英，成為國際人才
匯集中心\n\n\n\n1\n\n打造2030雙語國家\n\n更進一步接軌國際\n\n提升我國人才與產業的國際競爭力\n\n提升國人雙語力及數位
力\n\n培育專業領域的雙語人才\n\n處理國際組織及使用英語國家之涉外業務公務人員\n\n國發會\x0b統合協調\n\n教育部等相關
部會\n\n教育體系\n\n非教育體系\n\n(教育部)\n\n(國發會)\n\n成立行政法人專責推動\n\n以重點培育及普及提升為\x0b推動主
軸，提升學生英語力\n\n運用數位科技提升英語力\n\n增加英語師資擴充英語教學能量\n\n雙語資料庫\n\n學習資源網\n\n偏遠地
區\n\n數位學習資源\n\n國際學習資源\n\n強化學校數位學習軟硬體量能\n\n設立國際影音頻道\n\n(教育部、通傳會、科會辦、國
發會)\n\n延攬外國專業人才來臺\n\n成立專業領域英語學院\n\n(文化部)\n\n擴增英語檢測量能\n\n(教育部、金管會、文化部)\n
\n提升涉外業務\x0b公務人員英語力\n\n(考選部、人事行政總處)\n\n13\n\n\n\n2\n\n積極培育本土數位人才\n\n本會協同相關部
會研擬「關鍵人才培育及延攬方案(草案)」，俟報院核定後，協調相關部會全力落實推動，提升國家競爭力\n\n\n\n3\n\n致力延攬
國際專業人才\n\n推動外國人才專法修法\n\n109.11.14 完成預告\n\n110.1.29 
陳報行政院\n\n鬆綁工作條件，爭取優秀外國大學畢業生等\n\n放寬居留及依親規定，縮短申請永居年限等\n\n優化社會保障，增
加租稅、健保等誘因\n\n內容\n\n進度\n\n實施攬才配套，發揮外國人才專法推動綜效\n\n109.10成立就業金卡辦公室，提供一條
龍專案服務\n\n整合就業PASS卡及創業家簽證，納入就業金卡線上  
申辦平台\n\n推動2030雙語國家政策，以及完善海外人才子女在臺教育環境(教育部)\n\n鎖定我國所需人才，  
精準全球延攬\n\n協助國際人才對接我國產業(經濟部、科技部等)\n\n強化吸引外國學生      
來

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n"],
    chunk_size=150,
    chunk_overlap=0)
splits = text_splitter.split_documents(ppt_docs)
for i in splits[:5]:
    print(i.page_content)
    print('_'*10)

立法院第10屆第3會期經濟委員會

國家發展委員會業務報告

國家發展委員會

主任委員 龔明鑫



全球經濟恢復成長

國際機構全球經濟成長率預測

單位：%

2021年1月預測

2021年2月預測

2021年3月預測
__________
經濟風險仍高：疫情尚難緩和、美中衝突長期化、全球債務攀升金融風險升高、不均衡復甦



2021年展望

實質輸出成長+5.05%

全球貿易量回升

數位轉型商機

新興科技應用需求仍殷

2021年(f)  (+4.64%)

單位：%

近7年新高!!

實質民間消費成長+3.74%
__________
國內就業市場穩定

國內外股市活絡

2020年(p)  (+3.11%)

實質固定投資成長+3.50%

半導體、5G及綠能投資

臺商回臺三大方案

註： (p)表初步統計數，(f)表預測數。

資料來源：行政院主計總處，2021年2月20日。



施政方向與重點

創新導向，
__________
成為全球經濟關鍵力量

延攬精英，

成為國際人才匯集中心

精進治理，

落實區域永續平衡發展

數位政府，

提升全方位施政效能



創新導向，成為全球經濟關鍵力量



1

協調推動六大核心戰略產業

本會、經濟部、科技部等18個單位共同推動，讓臺灣成為全球經濟的關鍵力量
__________
世界經濟論壇(WEF)107、108連續兩年評比

亞洲‧矽谷1.0重要成果

2-1

臺灣全球4大超級創新國之一

積極投資新創

物聯網成為兆元產業

加碼匡列

20億→50億

已通過131家企業， 

投資19.28億、

帶動投資50.48億元

形塑國家新創品牌
__________


In [ ]:
def rag(splits, name):
    db = FAISS.from_documents(splits, embeddings)
    db.save_local(folder_path="/content/drive/MyDrive/office_db",
                  index_name=name)
    new_db = FAISS.load_local(
        folder_path="/content/drive/MyDrive/office_db",
        index_name=name,
        embeddings=embeddings,
        allow_dangerous_deserialization=True)
    retriever=new_db.as_retriever()
    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | chat_model
        | str_parser
    )
    return chain

In [ ]:
ppt_chain = rag(splits, 'ppt')

In [ ]:
print(ppt_chain.invoke("台灣的創新產業如何?"))

根據提供的文件內容來看，台灣的創新產業發展相當活躍。台灣政府積極推動新創產業發展，通過擴大投融資、加速新創成長、形塑國家新創品牌、深化物聯網國際夥伴關係等策略來推動創新產業的發展。此外，台灣已成為全球經濟關鍵力量之一，延攬精英、成為國際人才匯集中心、提升施政效能等措施也有助於創新產業的蓬勃發展。综合來看，台灣的創新產業發展前景看好。


### Word

In [ ]:
word_docs = office_file('/content/example_word.docx')
pprint(word_docs)

[
    Document(
        metadata={'source': '/content/example_word.docx'},
        page_content='常見詐騙手法話術解析與預防策略\n\n第一章\u3000假網拍詐騙\n\n手法及話術解析：\n\n民眾透過臉書
、LINE或知名拍賣網站從事網路購物，詐騙集團便利用當前最新款的3C產品、限量球鞋、名牌包或熱門演唱會門票等，以明顯低於
市價之價格誘引民眾下單並要求以LINE或Messenger私下交易。等被害人匯款後卻不出貨，且失去聯繫；或以貨到付款方式取貨開箱
後，才發現是劣質商品。\n\n預防策略\n\n網購商品應慎選優良有信用之網路商家，透過面交方式或選擇提供第三方支付之網購平
臺，以保障雙方權益並減少消費糾紛。\n\n避免透過LINE、Messenger等通訊軟體與賣家私下聯繫交易，以免求助無門。\n\n應加強
查核網路商家的真實性，如粉絲專頁成立時間短、粉絲或追蹤人數過少等，可立即至經濟部網站查詢公司名稱、地址等基本資料。\
n\n第二章\u3000假投資*新增\n\n手法及話術解析\n\n詐騙集團透過網路社群或交友軟體主動認識被害人，並假借股票、虛擬通貨
、期貨、外匯及基金等名義，吸引民眾加入LINE投資群組，初期會先讓民眾小額獲利，再以資金越多獲利越多說詞，引誘民眾加入
投資網站或下載APP並投入大量資金，後續再以洗碼量不足、繳保證金、IP異常等理由拒絕出金，民眾發現帳號遭凍結或網站關閉才
發現遭詐。\n\n預防策略\n\n高獲利必定伴隨高風險，聽到「保證獲利」、「穩賺不賠」必定是詐騙，民眾應選擇自己有深入瞭解
過的投資標的，不要輕信來源不明投資管道或網路連結，如遇到加「LINE」進行投資必為詐騙。\n\n「165防騙宣導」LINE官方帳號
及165全民防騙網可查詢詐騙LINE 
ID、假投資(博弈)網站及詐騙電話，同時與「165全民防騙」臉書粉絲專頁每週公布投資詐騙網站，提供民眾即時掌握最新詐騙訊息
，或可至投信投顧公會網站(www.sitca.org.tw)查詢合法投資管道。\n\n\n\n第三章\u3000ATM解除分期付款詐騙\n\n手法及話術解
析\n\n詐騙集團攻擊資安防護不足電子商務平臺或基金會之後臺資料庫，取得民眾的交易紀錄與個資，再假冒商家或基金會與銀行
來電，謊稱因設定錯誤，將重複扣款，要求民眾操作ATM、網路銀行或至超商購買遊戲點數進行解除設定。民眾依照指示操作後，發
現帳戶餘額減少，才知道上當受騙。\n\n預防策略\n\n接獲陌生來電或境外來電(開頭有+)必須提高警覺，不要因為能明確說出交易
明細而相信。\n\n接到來電以「誤設為重複扣款」、「升級VIP會員」、「系統出錯」等詐騙關鍵字就要提高警覺，應將電話掛斷後
，自行查詢客服電話並撥打確認，或撥打165反詐騙諮詢專線諮詢。\n\nATM只能「提款」與「轉帳」功能，不能解除分期付款或退
款；遊戲點數亦只有儲值遊戲之功能，並無法解除任何設定或身分識別。\n\n第四章\u3000假愛情交友*新增\n\n手法及話術解析\n
\n詐騙集團透過社群網站或交友軟體等管道隨機加好友，並盜用網路上帥哥、美女照片，假冒戰地軍官、軍醫、服務於聯合國官員
，讓民眾誤入情網，再以境外寄送跨境包裹、禮物需要關稅或儲存結婚基金介紹投資管道等方式詐騙民眾匯款，直到遭對方封鎖才
知道受騙。\n\n預防策略\n\n接獲陌生人要求加好友，切勿在言談間透露太多個人資訊，避免個人資訊遭盜用。\n\n沒見過面的網
友要求匯錢時，務必提高警覺，可要求開視訊聯繫通話確認本人身份，勿輕易將金錢交付他人。\n\n第五章\u3000猜猜我是誰\n\n
手法及話術解析\n\n詐騙集團假冒「兒女」、「親戚」或「多年不見的友人」，透過電話或通訊軟體(如LINE)與民眾聯繫，並以換
過手機所以不是原來的號碼、暱稱等方式要求民眾加入新手機號碼或LINE 
ID，過幾天後再以臨時急用、投資\n\n\n\n周轉等理由詐騙民眾匯款，事後民眾聯絡到當事人確認無此事後才知道被騙。\n\n預防
策略\n\n接到以親友名義來電借錢，務必以舊有電話向當事人親自確認，切勿貿然前往銀行匯款，以免遭詐騙。\n\n提供的匯款帳
戶名稱與親友本人名字不同，詐騙的可能性將大幅提高，更應主動查證。\n\n可與親友約定專屬密語，做為懷疑對方身分時的確認
。\n\n第六章\u3000假冒機構(公務員)\n\n手法及話術解析\n\n詐騙集團假冒健保局、醫院及中華電信或其他公務機關撥打市內電
話予民眾，用語音的方式引導被害人轉接客服人員，並以被害人遭冒用身分辦理開戶、請領健保補助或申辦門號為由，主動協助轉
接警察機關製作筆錄，另告知涉及洗錢或其他刑事案件，強調偵查不公開勿告知親友，再要求民眾至超商收取傳真或以LINE傳送法
院偽造公文取信被害人，須被害人配合監管帳戶等話術行騙得逞。\n\n預防策略\n\n聽到「遭冒用身分」、「偵查不公開」、「接
收法院偽造公文」、「監管帳戶」等關鍵字，就是假冒機構(公務員)詐騙。\n\n接獲陌生來電告知涉入刑案或收到疑為政府機關公
文書，應小心求證，先詢問對方單位、職稱、姓名等，掛斷電話後再向該單位求證。\n\n警察機關不會在電話中做筆錄，檢警調等
單位處理司法案件，會以通知書通知涉案人到案說明或至指定機關繳交款項。\n\n第七章\u3000假求職*新增\n\n手法及話術解析\n
\n於報章雜誌或網路訊息刊登求職訊息誘騙民眾，再以設定薪資轉帳等名義騙取被害人之金融帳戶及密碼，續遭詐騙集團作為人頭
帳戶之用。\n\n刊登求職廣告向應徵者騙取訂金、保證金、治裝費、訓練費或替代保證金。\n\n利用社群網站或交友軟體刊登出國
高薪打工等不實廣告，利用面試時機強行帶往銀行開戶並設定約定轉帳啟用網路銀行等功能，或以簽約出借帳戶或\n\n\n\n給付每
日報酬為由，控制被害人人身自由，拘禁於民宅或旅館內，藉此取得並利用被害人銀行帳戶收取詐騙款項。\n\n預防策略\n\n合法
公司徵才求職不會透過網路訊息貼文，更不會以私人帳號發訊息。\n\n切勿聽從指示申辦個人約定帳戶，對於個人銀行帳戶、網路
銀行帳號密碼等，應自行妥善保管，絕對不可交付陌生人。\n\n對網路徵才公司多查、多問、多存疑，可去電確認真偽，面試前先
告知家人朋友行蹤，簽訂工作合約前務必詳閱內容，如有疑慮的契約不要簽，以保護自身安全。\n\n如若自身或家人遇有類似情形
，請即向我國各駐外領使館處尋求協助或撥打急難救助電話0800-085095，並儘速就近向警察機關報案\n\n第八章\u3000盜(冒)用網
路帳號\n\n手法及話術解析\n\n詐騙集團盜取民眾LINE、Facebook等社群軟體帳號，再以盜用帳號傳訊息給其親友：\n\n朋友參加
攝影比賽，麻煩幫忙投票WWW.XXXX.COM？\n\n手機送修，麻煩提供手機號碼，以便接收簡訊認證碼…\n\n剛才出車禍急需用錢，可以
匯款給我嗎？\n\n現在不方便繳錢或匯款，可以幫忙嗎？\n\n現在不方便買點數卡，可以幫忙買張點數卡嗎？\n\n預防策略\n\n當
朋友傳送訊息要求協助匯款或代購遊戲點數時，務必提高警覺，撥通電話再行確認。\n\n多組帳號勿使用同組密碼，避免因其中一
組帳號遭破解而其他帳號都被盜用，並應定期更改密碼。\n\n避免在公用電腦登入私人帳號密碼。\n\n如果訊息中帶有不明連結，
請先向發送訊息的朋友確認。\n\n勿下載來源不明或非官方認證應用程式，以免個資外洩。'
    )
]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["第[一二三四五六七八九十]+章"],
    is_separator_regex=True,
    chunk_size=200,
    chunk_overlap=10)
splits = text_splitter.split_documents(word_docs)
for i in splits[:3]:
    print(i.page_content)
    print('_'*10)

常見詐騙手法話術解析與預防策略
__________
第一章　假網拍詐騙

手法及話術解析：

民眾透過臉書、LINE或知名拍賣網站從事網路購物，詐騙集團便利用當前最新款的3C產品、限量球鞋、名牌包或熱門演唱會門票等，以明顯低於市價之價格誘引民眾下單並要求以LINE或Messenger私下交易。等被害人匯款後卻不出貨，且失去聯繫；或以貨到付款方式取貨開箱後，才發現是劣質商品。

預防策略

網購商品應慎選優良有信用之網路商家，透過面交方式或選擇提供第三方支付之網購平臺，以保障雙方權益並減少消費糾紛。

避免透過LINE、Messenger等通訊軟體與賣家私下聯繫交易，以免求助無門。

應加強查核網路商家的真實性，如粉絲專頁成立時間短、粉絲或追蹤人數過少等，可立即至經濟部網站查詢公司名稱、地址等基本資料。


__________
第二章　假投資*新增

手法及話術解析

詐騙集團透過網路社群或交友軟體主動認識被害人，並假借股票、虛擬通貨、期貨、外匯及基金等名義，吸引民眾加入LINE投資群組，初期會先讓民眾小額獲利，再以資金越多獲利越多說詞，引誘民眾加入投資網站或下載APP並投入大量資金，後續再以洗碼量不足、繳保證金、IP異常等理由拒絕出金，民眾發現帳號遭凍結或網站關閉才發現遭詐。

預防策略

高獲利必定伴隨高風險，聽到「保證獲利」、「穩賺不賠」必定是詐騙，民眾應選擇自己有深入瞭解過的投資標的，不要輕信來源不明投資管道或網路連結，如遇到加「LINE」進行投資必為詐騙。

「165防騙宣導」LINE官方帳號及165全民防騙網可查詢詐騙LINE ID、假投資(博弈)網站及詐騙電話，同時與「165全民防騙」臉書粉絲專頁每週公布投資詐騙網站，提供民眾即時掌握最新詐騙訊息，或可至投信投顧公會網站(www.sitca.org.tw)查詢合法投資管道。




__________


In [ ]:
word_chain = rag(splits, 'word')

In [ ]:
print(word_chain.invoke("如何防治詐騙電話?"))

根據資料中提供的內容，防治詐騙電話的方法包括：小心求證陌生來電的身份和目的，並向對方提問相關單位、職稱、姓名等資訊後再確認；警察機關不會在電話中做筆錄或要求民眾至超商收取傳真，應提高警覺；接到以親友名義來電要求匯款，應以原有電話向當事人親自確認，切勿貿然匯款。此外，務必多組帳號使用不同密碼，避免帳號被盜用。當朋友傳送訊息要求匯款或代購遊戲點數時，也應提高警覺，撥通電話再行確認。


### Excel

In [ ]:
excel_docs = office_file('/content/example_excel.xlsx')
pprint(excel_docs)

[
    Document(
        metadata={'source': '/content/example_excel.xlsx'},
        page_content='項次 項目 詳細說明 1 營運時間 
1.週二至過五09:00~17:00。2.寒暑假期間及連續假期(收假日除外)延長營運至20:00。3.週六、週日或收假日延長營運至18:00。4.
週一(寒暑假期及連續假期除外)及農曆除夕為休園日不對外開放。5.營運時間如有調整將另行公告。 2 門票資訊 
全票：30元、優待票：15元、團體票：30人以上按票價打7折;◎全票：一般民眾◎優待票：1.年滿7歲而未滿12歲之兒童(出示身分證
件);2.持有效學生證之在學學生(外國在學學生須持有效期之國際數位學生證);3.持有臺北市政府核發之低收入戶證明;◎免費入園：
1.6歲(含)以下兒童;2.設籍臺北市之55歲(含)以上原住民;3.65歲(含)以上長者;4.臺北市退休公教人員持退休證者;5.身心障礙者及
其必要之陪伴者1人;6.持志願服務榮譽卡之志工;7.持臺北市國小數位學生證者(私立學校為一般學生證);8.週六及連續假期(連續假
期最後一天除外)16:00後入園;9.平常日凡30人(含)以上之團體，依現行團體預約方式，於前1日17:00前完成預約作業;10.設籍臺北
市之市民，於週三入園;11.持有臺北市之義勇人員(含交通、警察、消防)及民防人員身分證明。 3 K2小型遊樂設施介紹 
鋼鐵碰碰車單次票價30元、身高未達90公分兒童或行動不便者請勿搭乘；每輛車限乘1人，限制體重60公斤以下之人員乘坐。 4 
1號遊樂設施介紹 
海洋總動員單次票價20元、身高未達100公分兒童必須由成人陪同。◎針對身高未滿85公分之幼童，或身高超過85公分但未滿2歲之幼
童(須出示證明文件)，在親友陪同下可免費搭乘遊具，親友仍需付費。 5 2號遊樂設施介紹 
水果摩天輪單次票價30元、身高未達120公分兒童必須由成人陪同，身高超過195公分請勿搭乘。◎針對身高未滿85公分之幼童，或身
高超過85公分但未滿2歲之幼童(須出示證明文件)，在親友陪同下可免費搭乘遊具，親友仍需付費。 6 3號遊樂設施介紹 
歐彼特菈星空號(銀河號)單次票價20元、身高未達100公分兒童必須由成人陪同。◎針對身高未滿85公分之幼童，或身高超過85公分
但未滿2歲之幼童(須出示證明文件)，在親友陪同下可免費搭乘遊具，親友仍需付費。 7 4號遊樂設施介紹 
飛天神奇號單次票價20元、身高未達90公分兒童請勿搭乘，身高達90公分未滿100公分之兒童，必須由成人陪同。 8 
5號遊樂設施介紹 宇宙迴旋單次票價20元、身高未達90公分兒童請勿搭乘，身高達90公分未滿100公分之兒童，必須由成人陪同。 9
6號遊樂設施介紹 星空小飛碟單次票價20元、身高未達90公分兒童及超過190公分之成人請勿搭乘 
；身高達90公分未滿100公分之兒童，必須由成人陪同。 10 7號遊樂設施介紹 
轉轉咖啡杯單次票價20元、身高未達90公分兒童請勿搭乘，身高達90公分未滿100公分之兒童，必須由成人陪同。 11 
8號遊樂設施介紹 巡弋飛椅單次票價20元、身高未達90公分兒童請勿搭乘，身高達90公分未滿120公分之兒童，必須由成人陪同。 
12 9號遊樂設施介紹 
尋寶船單次票價30元、身高未達110公分兒童請勿搭乘，身高達110公分未滿130公分之兒童，必須由成人陪同。 13 
10號遊樂設施介紹 
魔法星際飛車單次票價30元、身高未達110公分兒童請勿搭乘，身高達110公分未滿130公分兒童，必須由成人陪同。 14 
11號遊樂設施介紹 小飛龍單次票價20元、身高未達110公分兒童請勿搭乘，身高達110公分未滿130公分兒童，必須由成人陪同。 15
12號遊樂設施介紹 
幸福碰碰車單次票價30元、身高未達110公分兒童請勿搭乘，身高達110公分未滿130公分兒童，必須由成人陪同。 16 
13號遊樂設施介紹 
叢林吼吼樹屋單次票價30元、身高未達110公分兒童請勿搭乘，身高達110公分未滿140公分兒童，必須由成人陪同。 17 
寶貝叢林歷險館遊戲場介紹 1. 
每場次遊戲時間50分鐘，可容納人數60名，提供給不同年齡層孩童盡情遊戲遊玩。【各場次時間與入場規範調整，詳依本園區 
網頁及現場公告】。2. 
設施建議12歲以下兒童遊玩。未滿6歲（或120公分以下）之兒童須由成人陪同入場。每位成人最多可陪同10名兒童入場。3. 
票價：\\n(1) 兒童50元/次、成人30元/次。\\n(2) 臺北市敬老卡（限本人使用）適用於本場館使用，每場次最多可扣 30 點（1 
元 1點），如遇點數不足時將以卡片餘額抵扣。\\n(3) 採無現金交易方式付費，提供悠遊卡、一卡通、愛金卡、信用卡、LINE 
PAY、一卡通MONEY、悠遊付、街口支付、Pi 錢包等多元支付方式，歡迎多加利用。\\n4. 
符合下列身份者得免費入場。\\n(1)2歲以下(須出示證明文件)或身高未滿85公分之幼童，得由成人陪同免費入場，惟陪同之成人仍
須付費。\\n(2)身心障礙者本人，出示中華民國核發之身心障礙證明正本可免費入場，另註明註記有必要陪伴者資格者，陪伴者1人
得免費入場。\\n5.本場館另提供學校、企業及各團體包場優惠方案，意者請聯絡(02)2833-3823 分機 622 
翁小姐洽詢。\\n★注意事項 
\\n1.為維護場內遊憩兒童安全及衛生，除進、離場走道及休憩區外，進入兒童遊戲區域內請先脫鞋，並穿著襪子遊玩。 
\\n2.每場次結束後，將進行場內之互動或觸控設施清潔消毒，敬請配合清場。【場次時間與入場規範調整，詳依現場公告】 18 
沙坑遊戲場介紹 1. 
位於奇幻森林與夢想海區之地面層，包含沙坑、融合式遊戲設施及滑梯，讓小朋友免費體會玩沙、融合式遊戲設施及滑梯的樂趣。2
. 
造型滑梯區基於安全考量僅提供身高90公分至150公分以內兒童使用；融合式遊戲設施(玩沙盤、玩水盤提供2-12歲兒童使用，轉轉
盤限定6歲以上且須有大人陪同方可使用)；沙坑區則沒有身高及年齡限制。3. 
兒童遊玩時請勿潑沙或做出危險動作，並須有20歲(含)以上成人在旁陪同指導正確使用方法及維護安全。 19 投幣式遊戲機介紹 
1樓免費室內遊戲區左側及火山後方區域。2樓美食街右側區域、親子堡及劇場入口處。3樓碰碰車遊戲場右側區。 20 園區美食商店
1樓美食及商店：轉轉蛋樂園、智慧無人超商、快樂屋、烤畫趣、化石先生、智慧商店;2樓美食及商店：統一多拿滋、?鋪、萃茶風
、義品院義大利麵、四海遊龍、鴻屋蛋包飯、花蓮扁食、突厥、木槿花韓式食堂、呀咪食堂、熊大廚、統一超商、豆豆先生工作坊;
3樓美食及商店：漢堡王、土耳其冰淇淋、葡兒飛、大眼蝦、瓦非屋、貓貓小站、鼎豐FOOD 21 兒童劇場 
具有438個座位席並設有身心障礙平臺，由【如果兒童劇團】經營，每月定期更換演出劇目。在團長趙自強的帶領下，尊重孩子本能
的思考與創造力，讓懂得欣賞表演藝術的孩子能在生活中感動、學習。我們深信，戲劇中的勇氣、樂觀、幽默……能夠陪伴孩子，成
為他們人生中神奇的金鑰匙。讓所有進劇場看戲的小朋友都能帶回一點歡樂、感動、回憶，創造親子美好時光。票價資訊： 
◎演出票價：100元;◎購票地點：（1）兒童新樂園兒童劇場入口處;（2）如果兒童劇場售票系統網站;◎付款方式： 
（1）園內購票→可使用現金及悠遊卡;（2）網路購票→刷卡 
;◎優惠方案規劃：（1）3歲以下兒童免費入場。（2）55歲以上，購票得享5折優待。（3）身心障礙人士及其必要之陪同人員（限一
人），購票得享5折優待，並憑票券及身心障礙手冊入場，未出示手冊者需補足差額始得入場。\\n（4）團體購票另有優惠，請洽：
02-28314567。◎退換票與注意事項：（1）為尊重大小觀眾權益，不論大人小孩均需持票入場。（2）退、換票最遲請於演出前10日
辦理，並酌收票價10%手續費，逾期恕不受理。（3）遲到及中途離席的觀眾，請依主辦單位安排適當時間入場。（4）票券為無記名
之有價券，請持票人妥善保管票券，如發生遺失、破壞、燒毀或無法辨識等情形，一概不重新發票。（5）請留意票券演出日期及時
間，如有過期情事，恕不接受退款或更換。 22 樹蛙表演場 
一座提供入園遊客免費觀賞藝文活動之戶外表演場地。為活絡場地使用及擴大民眾參與，提供場地予個人、學校、公司行號、機關
團體等單位申請演出 
，表演項目有戲劇、默劇、丑劇、舞蹈、歌唱、演奏、魔術、民俗技藝、雜耍、偶戲、朗誦詩文集、行動藝術等。 23 
大眾運輸轉乘資訊 捷運轉乘公車：◎捷運劍潭站出口3→公車轉乘站 41，紅30，529(例假日停駛)，市民小巴８，兒樂2號線 
→兒童新樂園◎捷運士林站出口1→公車轉乘站 255區，紅30，兒樂1號線(平日停駛)→兒童新樂園◎捷運芝山站出口1→公車轉乘站 
兒樂１號線（平日停駛)=>兒童新樂園;註：兒童新樂園直達公車「兒樂1號線」及「兒樂2號線」為一般聯營公車路線（配合兒童新
樂園開放時間行駛）並以1段票收費，行駛路線圖：（相關資訊以臺北市公共運輸處公告為主，請洽「我愛巴士5284臺北市公車資訊
網」查詢）;◎捷運士林站出口1→公車轉乘站→紅12、557→國立科教館站下車，步行至兒童新樂園(步行約4分鐘)。◎捷運士林站出口1→
公車轉乘站→北環幹線(原620)→士林監理站下車，步行至兒童新樂園(步行約3分鐘)。捷運轉乘公共自行車(YouBike)：◎捷運劍潭站
出口2 YouBike租

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n"],
    chunk_size=150,
    chunk_overlap=0)
splits = text_splitter.split_documents(excel_docs)
for i in splits[:5]:
    print(i.page_content)
    print('_'*10)

項次 項目 詳細說明 1 營運時間 1.週二至過五09:00~17:00。2.寒暑假期間及連續假期(收假日除外)延長營運至20:00。3.週六、週日或收假日延長營運至18:00。4.週一(寒暑假期及連續假期除外)及農曆除夕為休園日不對外開放。5.營運時間如有調整將另行公告。 2 門票資訊 全票：30元、優待票：15元、團體票：30人以上按票價打7折;◎全票：一般民眾◎優待票：1.年滿7歲而未滿12歲之兒童(出示身分證件);2.持有效學生證之在學學生(外國在學學生須持有效期之國際數位學生證);3.持有臺北市政府核發之低收入戶證明;◎免費入園：1.6歲(含)以下兒童;2.設籍臺北市之55歲(含)以上原住民;3.65歲(含)以上長者;4.臺北市退休公教人員持退休證者;5.身心障礙者及其必要之陪伴者1人;6.持志願服務榮譽卡之志工;7.持臺北市國小數位學生證者(私立學校為一般學生證);8.週六及連續假期(連續假期最後一天除外)16:00後入園;9.平常日凡30人(含)以上之團體，依現行團體預約方式，於前1日17:00前完成預約作業;10.設籍臺北市之市民，於週三入園;11.持有臺北市之義勇人員(含交通、警察、消防)及民防人員身分證明。 3 K2小型遊樂設施介紹 鋼鐵碰碰車單次票價30元、身高未達90公分兒童或行動不便者請勿搭乘；每輛車限乘1人，限制體重60公斤以下之人員乘坐。 4 1號遊樂設施介紹 海洋總動員單次票價20元、身高未達100公分兒童必須由成人陪同。◎針對身高未滿85公分之幼童，或身高超過85公分但未滿2歲之幼童(須出示證明文件)，在親友陪同下可免費搭乘遊具，親友仍需付費。 5 2號遊樂設施介紹 水果摩天輪單次票價30元、身高未達120公分兒童必須由成人陪同，身高超過195公分請勿搭乘。◎針對身高未滿85公分之幼童，或身高超過85公分但未滿2歲之幼童(須出示證明文件)，在親友陪同下可免費搭乘遊具，親友仍需付費。 6 3號遊樂設施介紹 歐彼特菈星空號(銀河號)單次票價20元、身高未達100公分兒童必須由成人陪同。◎針對身高未滿85公分之幼童，或身高超過85公分但未滿2歲之幼童(須出示證明文件)，在親友陪同下可免費搭乘遊具，親友仍需付費。 7 4號遊樂設施介紹 飛天神奇號單次票價20元、身高未達90公分兒童請勿搭乘，身高達90公分未滿100公分之兒童，必須由成人陪同

In [ ]:
excel_chain = rag(splits, 'excel')

In [ ]:
print(excel_chain.invoke("我身高超過 120 公分, 我可玩那些遊樂設施?"))

根據提供的資訊，您身高超過120公分可參加以下遊樂設施：
- 1 號遊樂設施：海洋總動員
- 2 號遊樂設施：水果摩天輪
- 3 號遊樂設施：歐彼特菈星空號(銀河號)
- 9 號遊樂設施：尋寶船
- 10 號遊樂設施：魔法星際飛車
- 13 號遊樂設施：叢林吼吼樹屋

這些遊樂設施都適合身高超過120公分的您玩樂。


### CSV

In [ ]:
import pandas as pd
df = pd.read_csv("/content/example_csv.csv", skiprows=1)
df.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/03/01,"24,167,721","16,699,995,060",697.0,697.0,688.0,689.0,-1,"26,282"
1,113/03/04,"97,210,112","69,868,348,694",714.0,725.0,711.0,725.0,36,"125,799"
2,113/03/05,"73,299,411","53,751,887,376",735.0,738.0,728.0,730.0,5,"69,851"
3,113/03/06,"52,464,833","38,203,868,985",718.0,738.0,717.0,735.0,5,"49,897"
4,113/03/07,"80,382,406","61,221,034,146",755.0,769.0,754.0,760.0,25,"96,348"


In [ ]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.4 MB/s eta 0:00:00


LangChain 2.0 要求 `create_pandas_dataframe_agent` 必須明確指定 `allow_dangerous_code=True`, 否則會出現錯誤：

```
ValueError: This agent relies on access to a python repl tool which can execute arbitrary code. This can be dangerous and requires a specially sandboxed environment to be safely used. Please read the security notice in the doc-string of this function. You must opt-in to use this functionality by setting allow_dangerous_code=True.For general security guidelines, please see: https://python.langchain.com/v0.2/docs/security/
```

In [ ]:
from langchain_experimental.agents import (
    create_pandas_dataframe_agent)

agent = create_pandas_dataframe_agent(llm=chat_model,
                    df=df,
                    prefix='回答請使用繁體中文',
                    agent_type="openai-tools",
                    # langchain 2.0 要求加上這一行
                    allow_dangerous_code=True,
                    verbose=True)

In [ ]:
result = agent.invoke({"3月收盤價的平均值是?"})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['收盤價'].mean()"}`


762.71428571428573月收盤價的平均值是762.71。

> Finished chain.
3月收盤價的平均值是762.71。


In [ ]:
print(df['收盤價'].mean())

762.7142857142857


[Replit 專案](https://replit.com/@flagtech/Officefilebot)

[GitHub 可讀圖片 PDF 的 Source code](https://github.com/FlagTech/F4763)